In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
# Import necessary libraries
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOpenAI


#File paths for the PDF documents
pdf_paths = [
    "/home/habte/Downloads/Raptor Contract.docx.pdf",
    "/home/habte/Downloads/Raptor Q&A2.docx.pdf",
    "/home/habte/Downloads/Robinson Advisory.docx.pdf",
    "/home/habte/Downloads/Robinson Q&A.docx.pdf"
    ]

# Load PDF and split into tokens for all documents
pdf_data = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    pdf_data.extend(loader.load())

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
split_data = text_splitter.split_documents(pdf_data)

# Set up ChromaDB with vector embeddings
collection_name = "contracts_collection"
local_directory = "contracts_vect_embedding"
persist_directory = os.path.join(os.getcwd(), local_directory)

openai_key = os.environ.get('OPENAI_API_KEY')  
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
vect_db = Chroma.from_documents(
    split_data,
    embeddings,
    collection_name=collection_name,
    persist_directory=persist_directory
)
vect_db.persist()

# Initialize the OpenAI language model
chat_model = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_qa = ConversationalRetrievalChain.from_llm(
    chat_model,
    vect_db.as_retriever(),
    memory=memory
)

/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
# Example manual evaluation and debugging
user_questions = [
    "What are the terms of payment in the contract?",
    "Can you provide information about the termination clause?",
    "Is there any clause regarding intellectual property rights?"
]

In [4]:
for question in user_questions:
    response = chat_qa({"question": question, "chat_history": []})
    print("User:", question)
    print("RAG System Response:", response["answer"])
    print("Debugging Information:")
    

/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


User: What are the terms of payment in the contract?
RAG System Response: The terms of payment in the contract include:
1. Fees of $9 per hour up to a monthly limit of $1,500.
2. Workspace expense of $100 per month.
3. Other reasonable and actual expenses if approved by the company in writing and in advance.
Debugging Information:
User: Can you provide information about the termination clause?
RAG System Response: The termination notice period in the contract is 14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of the Agreement.
Debugging Information:
User: Is there any clause regarding intellectual property rights?
RAG System Response: Yes, there are several clauses regarding intellectual property rights in the contract. These clauses include information about the infringement of Company IP, IP Contracts, Title to Company Intellectual Property, Sufficiency of rights t

In [5]:
user_questions = [
    "What are the terms of payment in the contract?",
    "Can you provide information about the termination clause?",
    "Is there any clause regarding intellectual property rights?"
]

for question in user_questions:
    response = chat_qa({"question": question, "chat_history": []})
    print("User:", question)
    print("RAG System Response:", response["answer"])
    print("Debugging Information:")
   


User: What are the terms of payment in the contract?
RAG System Response: The clauses regarding intellectual property rights in the contract are detailed in sections (e), (f), (b), (c), and (d). These sections cover topics such as infringement of company intellectual property, title to company intellectual property, sufficiency of intellectual property rights, and infringement of third party intellectual property.
Debugging Information:
User: Can you provide information about the termination clause?
RAG System Response: The termination notice period in the contract is 14 days for convenience by both parties. The Company may terminate without notice if the Advisor refuses or cannot perform the Services or is in breach of any provision of the Agreement.
Debugging Information:
User: Is there any clause regarding intellectual property rights?
RAG System Response: Yes, there are clauses in the contract regarding intellectual property rights. The contract discusses various aspects of intelle

In [6]:
# Initialize a separate language model for LLM-assisted evaluation
from langchain_community.chat_models import ChatOpenAI

separate_llm = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo")

for question in user_questions:
    # Generate response using the RAG system
    rag_response = chat_qa({"question": question, "chat_history": []})

    # Generate response using the separate language model
    separate_llm_response = separate_llm.invoke(question)  # Using the 'invoke' method to generate response

    # Print the user's question and both responses for comparison
    print("User:", question)
    print("RAG System Response:", rag_response["answer"])
    print("Separate LLM Response:", separate_llm_response)
    # Compare the responses and identify discrepancies


User: What are the terms of payment in the contract?
RAG System Response: The clauses regarding intellectual property rights in the contract include:
- Infringement of Company IP (Section e)
- IP Contracts (Section f)
- Title to Company Intellectual Property (Section b)
- Sufficiency (Section c)
- Infringement of Third-Party IP (Section d)

These clauses cover aspects such as ownership, infringement, licensing, contractual obligations, and the rights and obligations related to intellectual property within the context of the contract.
Separate LLM Response: content='The terms of payment in the contract are as follows:\n\n- Payment shall be made in full within 30 days of the invoice date.\n- Payment can be made via bank transfer or cheque.\n- Late payment will incur a 10% penalty fee.\n- Any disputes regarding payment must be resolved within 14 days of notification.\n- The currency of payment shall be in US dollars.'
User: Can you provide information about the termination clause?
RAG Sys